## Cross-validation of sparse CCA

Predict expression of methylotrophy genes from methanotrophy genes.  Previously pooled on species.

First, run: ../notebooks/prepare_data/prepare_X_m_expression_Y_nmm_expression_Xval.ipynb

That fills /Users/janet/ML_meta-omics/data/cross_val_data

In [ ]:
import pandas as pd

In [ ]:
import matplotlib as mpl
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import subprocess
import sys

In [ ]:
sys.path.append('../../code/')

from CCA import CcaAnalysis, ExpressionCCA

In [ ]:
! ls ../../data/cross_val_data/

In [ ]:
raw_data_path = '../../data/cross_val_data/'
xval_filenames = os.listdir(raw_data_path)
xval_filenames = [f for f in xval_filenames if '.tsv' in f]  # get rid of non-TSV files

In [ ]:
xval_filenames

In [ ]:
xval_filenames[0]

In [ ]:
def find_Xval_group(string):
    m = re.search('[_A-z]+fold([0-9]+)[._A-z]+', string)
    return int(m.group(1))

In [ ]:
find_Xval_group(xval_filenames[0])

In [ ]:
groups = list(set([find_Xval_group(f) for f in xval_filenames]))
groups

In [ ]:
xval_filenames

In [ ]:
[f for f in xval_filenames if 'methanotroph' in f and "fold3" + "_train" in f]

In [ ]:
def get_filename(string, fold, train=True):
    if train:
        suffix = '_train'
    else:
        suffix = '_val'
    filename_candidates = [f for f in xval_filenames
                           if string in f and "fold" + str(fold) + suffix in f]
    # get rid of u and v filenames
    #filename_candidates = [f for f in filename_candidates if not '_u' in f]
    #filename_candidates = [f for f in filename_candidates if not '_v' in f]
    assert len(filename_candidates) == 1, "file names found: {}".format(filename_candidates)
    return filename_candidates[0]

In [ ]:
get_filename('methanotroph', 1, train=True)

In [ ]:
get_filename('methanotroph', 3, train=True)

In [ ]:
get_filename('methylotroph', 3, train=True)

In [ ]:
summary = pd.DataFrame()

for fold in groups:
    print('fold: {}'.format(fold))
    row = {}
    row['k'] = fold
    row['m filename, train'] = get_filename('methanotroph', fold, train=True)
    row['m filename, val'] = get_filename('methanotroph', fold, train=False)
    row['nmm filename, train'] = get_filename('methylotroph', fold, train=True)
    row['nmm filename, val'] = get_filename('methylotroph', fold, train=False)
    #row['train x'] = np.genfromtxt(raw_data_path + row['m filename, train'], delimiter='\t', skip_header=1)
    #row['train z'] = np.genfromtxt(raw_data_path + row['nmm filename, train'], delimiter='\t', skip_header=1)
    #row['val x'] = np.genfromtxt(raw_data_path + row['m filename, val'], delimiter='\t', skip_header=1)
    #row['val z'] = np.genfromtxt(raw_data_path + row['nmm filename, val'], delimiter='\t', skip_header=1)
    
    # make a dataframe row
    row = {k:[v] for k, v in row.items()}
    df = pd.DataFrame.from_dict(row)
    #print(df)
    summary = pd.concat([summary, df], axis=0)
summary

### Run sparse CCA on each of these training sets

Start by demo of one set:

In [ ]:
summary.head(1)

In [ ]:
uv_dir = './crossvalidation_u_and_v_vectors/'
if not os.path.exists(uv_dir):
    os.mkdir(uv_dir)

In [ ]:
os.listdir(uv_dir)

In [ ]:
def run_CCA(summary, groups, run_R=True):
    results = {}  # make a dict of CCA objects
    for group in groups:
        fold_results = {}
        
        row = summary[summary['k'] == group]
        assert row.shape[0] == 1
        
        def prepare_filepath(set):
            filename = row[set][0]
            return os.path.join(raw_data_path, filename)
        
        x_filepath = prepare_filepath('m filename, train')
        z_filepath = prepare_filepath('nmm filename, train')
        
        penalty_x = 0.0335 # whole-data value
        penalty_z = 0.022  # whole-data value
        
        # todo: remove expected filename if it exsits.
        cca = ExpressionCCA(x_train_filename = x_filepath, 
                            z_train_filename = z_filepath,
                            x_val_filename = prepare_filepath('m filename, val'), 
                            z_val_filename = prepare_filepath('nmm filename, val'),
                            input_filepath = raw_data_path, 
                            u_v_output_dir = uv_dir,
                            penalty_x = 0.04,  # leads to thousands of nonzero weights in X-val
                            penalty_z = 0.03, # leads to thousands of nonzero weights in X-val 
                            path_to_R_script='../../code/sparse_CCA.R')
        
        results[group] = cca
    return results
    
results = run_CCA(summary, groups, run_R=False)

In [ ]:
results

In [ ]:
for n, c in results.items():
    print("--- {} ---".format(n))
    c.summarise()
    print(c.summary)
    c.plot_projections(filename='projection_for_set_{}.pdf'.format(n))